In [2]:
import tensorflow as tf
import numpy as np

In [3]:
g1 = tf.Graph()
with g1.as_default():
    a = tf.constant(5)
    b = tf.constant(6)
    c = tf.multiply(a,b)

In [4]:
with tf.Session(graph = g1) as sess1:
    output1 = sess1.run(c)
    replace_dict = {a:15}
    output2 = sess1.run(c,feed_dict = replace_dict)
    #print(output1,output2,a.eval())
    
    #Saving the graph in an object
    #writer = tf.summary.FileWriter('./TF1')
    #writer.add_graph(sess1.graph)
    #writer.close()

In [5]:
g2 = tf.Graph()
with g2.as_default():
    a = tf.placeholder(tf.int32,shape=[2,None])
    d = tf.add(a,a)
    B = tf.zeros([3,3,3])
    ones = tf.ones([3,3,3])
    uni = tf.random_uniform([3,3,3], minval = 0, maxval = 10)
    norm = tf.random_normal([3,3,3], mean = 5, stddev = 2)
    ran = tf.add(B,tf.add(ones,tf.add(uni,norm)))

In [6]:
with tf.Session(graph = g2) as sess2:
    input_dict = {a:np.array([[5,3],[2,1]],dtype = np.int32)}
    output1 = sess2.run(d,feed_dict = input_dict)
    output2 = sess2.run(ran)
    #print(output1,d,'\n\n',output2)

In [7]:
with tf.name_scope("Scope_A"):
    a = tf.Variable(0, name = 'a', trainable = True)
    b = tf.Variable(1, name = 'b', trainable = True)
with tf.name_scope("Scope_B"):
    c = tf.Variable(2, name = 'c', trainable = True)
    d = tf.Variable(3, name = 'a', trainable = True)
with tf.name_scope("Scope_C"):
    e = tf.add(a, c, name = 'output1')
    f = tf.add(b, d, name = 'output2')

g = tf.add(e,f)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
output = sess.run(g)
#print(output)

writer = tf.summary.FileWriter('./TF1')
writer.add_graph(sess.graph)
writer.close()

In [8]:
x = tf.Variable(1)
y = x + 1
init = tf.global_variables_initializer()
with tf.Session() as sess:
    #init.run() to initialize all the variables
    x.initializer.run()
    output1 = sess.run(y)
    output2 = y.eval()
    #print(output1,output2)

In [9]:
#x.graph is tf.get_default_graph()

In [10]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.Session() as sess:
    y_val,z_val = sess.run([y,z])
    #print(y_val)# 10
    #print(z_val)# 15

In [11]:
nsamples = 10000
nvar = 2;
nfact = 1;

F = np.zeros(shape=(nsamples,nvar),dtype = np.float64);
F[:,0] = np.random.random(nsamples);
F[:,1] = 2*F[:,0]

In [12]:
X = tf.constant(F[:,0], shape = [nsamples,1], name = "X",dtype = tf.float64)
y = tf.constant(F[:,1], shape = [nsamples,1], name = "y",dtype = tf.float64)
#XT = tf.transpose(X)
#theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
#with tf.Session() as sess:
    #theta_value = sess.run(theta)

In [13]:
n_epochs = 1000
learning_rate = 0.01
theta = tf.Variable(tf.cast(tf.Variable(tf.random_uniform([1, 1], -1.0, 1.0), name = "theta"), tf.float64))
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name = "mse")

gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            #print("Epoch", epoch, "MSE =", mse.eval())
            pass
        sess.run(training_op)     
    best_theta = theta.eval()
best_theta

array([[1.99648206]])

In [14]:
# Optimizers
n_epochs = 1000
learning_rate = 0.01
theta = tf.Variable(tf.cast(tf.Variable(tf.random_uniform([1, 1], -1.0, 1.0), name = "theta"), tf.float64))
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name = "mse")

#Momentum optimizer is faster 
optimizer1 = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum = 0.9)
optimizer2 = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer2.minimize(mse)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            #print("Epoch", epoch, "MSE =", mse.eval())
            pass
        sess.run(training_op)     
    best_theta = theta.eval()
best_theta

array([[1.99651397]])

In [15]:
# Placeholder values are used to pass values to tensorflow ops
A = tf.placeholder(tf.float32,shape = (None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict = {A:[[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict = {A:[[4, 5, 6], [7, 8, 9]]})

In [16]:
A = tf.constant(F[:,0], shape = [nsamples,1], name = "X",dtype = tf.float64)
B = tf.constant(F[:,1], shape = [nsamples,1], name = "y",dtype = tf.float64)
batch_size, m, n_epochs, learning_rate = nsamples/5, nsamples, 5, 0.01
n_batches = int(np.ceil(m/batch_size))
X1 = tf.placeholder(tf.float64, shape = [batch_size,1], name = "x")
y1 = tf.placeholder(tf.float64, shape = [batch_size,1], name = "y")
theta = tf.Variable(tf.cast(tf.Variable(tf.random_uniform([1, 1], -1.0, 1.0), name = "theta"), tf.float64))
y_pred = tf.matmul(X1, theta, name = "predictions")
error = y_pred - y1
mse = tf.reduce_mean(tf.square(error),name = "mse")
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)
init = tf.global_variables_initializer()

def fetch_batch(batch_index, batch_size):
    a_batch = tf.placeholder(tf.float64, shape = [batch_size,1], name = "x")
    b_batch = tf.placeholder(tf.float64, shape = [batch_size,1], name = "x")
    a_batch = A[batch_size * batch_index:batch_size * (batch_index + 1)].eval()
    b_batch = B[batch_size * batch_index:batch_size * (batch_index + 1)].eval()
    return a_batch, b_batch

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(batch_index, batch_size)
            sess.run(training_op, feed_dict = {X1:X_batch ,y1:y_batch})
        print(epoch)
    best_theta = theta.eval()
print(best_theta)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
[[2.]]


In [19]:
# Saving and restoring models(in case of computer crash)
#saver = tf.train.Saver()
#save_path = saver.save(sess, "/tmp/my_model.ckpt")
#with tf.Session() as sess:
    #saver.restore(sess, "/tmp/my_model_final.ckpt")
    #More code

In [22]:
# Sharing variables
#def relu(X, threshold):
    #with tf.name_scope("relu"):
        #[...]
        #return tf.maximum(z, threshold, name = "max")

#n_features = 2
#threshold = tf.Variable(0.0, name = "threshold")
#X = tf.placeholder(tf.float32, shape = (None, n_features), name = "X")
#relus = [relu(X, threshold) for i in range(5)]
#output = tf.add_n(relus,name = "output")

In [ ]:
#with tf.variable_scope("relu", reuse = True) as scope:
    #scope.reuse_variables()
    #threshold = tf.get_variable("threshold", shape = (), initializer = tf.constant_initializer(0.0))

In [23]:
#def relu(X):
    #with tf.variable_scope("relu",reuse = True):
        #threshold = tf.get_variable("threshold") # reuse existing variable
        #[...]
        #return tf.maximum(z, threshold, name = "max")

#X = tf.placeholder(tf.float32, shape = (None, n_features), name = "X")
#with tf.variable_scope("relu"):
    # create the variable
    #threshold = tf.get_variable("threshold", shape = (), initializer = tf.constant_initializer(0.0))
    #relus = [relu(X) for relu_index in range(5)]
    #output = tf.add_n(relus, name = "output")